In [1]:
import psutil
import serial
import serial.tools.list_ports
import time
import yaml

In [2]:
list(psutil.cpu_freq())

[3601.0, 0.0, 3601.0]

In [3]:
class RatePub:
    def __init__(self) -> None:
        self.__load__config()
        self.ser = self.__init__serial()
        self.usage = dict()
        self.max_cpu_freq = psutil.cpu_freq().max
        self.min_cpu_freq = psutil.cpu_freq().min

    def __init__serial(self) :
        port_list = list(serial.tools.list_ports.comports())
        if len(port_list) == 0:
            print("NO device connected")
        else:
            for port in port_list:
                print("avaliable com device: ",port.device)
        port = port_list[0]
        try:
            ser = serial.Serial(port.device, baudrate=self.baud_rate)
            print("open {} success with baudrate {}".format(port.device,self.baud_rate))
        except serial.SerialException as e:
            print(e)
        return ser 
    
    def __load__config(self):
        with open('publisher_config.yaml','r',encoding='utf-8') as f:
            cfg = yaml.load(f, Loader = yaml.FullLoader)
            self.publish_topic = cfg['publish_type']
            self.baud_rate = int(cfg['baud_rate'])

    def __get__usage(self):
        # must be 0~100
        self.usage["cpu_usage"] = float( psutil.cpu_percent() )
        self.usage["cpu_freq"] =  100 * float( psutil.cpu_freq().current/ self.max_cpu_freq)
        self.usage["mem_usage"] = float( psutil.virtual_memory().percent)
        psutil.cpu_freq()

    def run(self):
        try:
            while True:
                time.sleep(1)
                self.__get__usage()
                pub_val = self.usage[self.publish_topic] # float
                pub_val = str(round(2.55 * pub_val)).encode('utf-8')
                if self.ser.write(pub_val) != 0:
                    print("write {} {}".format(self.publish_topic,pub_val))
        except serial.SerialTimeoutException as e:
            print(e)

In [4]:
rate_pub = RatePub()
rate_pub.run()

avaliable com device: 

 COM4
open COM4 success with baudrate 115200
write mem_usage b'181'
write mem_usage b'182'
write mem_usage b'182'
write mem_usage b'183'
write mem_usage b'182'
write mem_usage b'182'
write mem_usage b'183'
write mem_usage b'182'
write mem_usage b'182'
write mem_usage b'177'
write mem_usage b'177'
write mem_usage b'177'
write mem_usage b'176'
write mem_usage b'176'
write mem_usage b'177'
write mem_usage b'177'
write mem_usage b'177'
write mem_usage b'175'
write mem_usage b'174'
write mem_usage b'174'
write mem_usage b'174'
write mem_usage b'174'
write mem_usage b'173'
write mem_usage b'173'
write mem_usage b'173'
write mem_usage b'172'
write mem_usage b'172'
write mem_usage b'172'
write mem_usage b'172'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_usage b'171'
write mem_us

KeyboardInterrupt: 